In [1]:
from bs4 import BeautifulSoup

import pandas as pd

import numpy as np

import requests

import re

from pandas import DataFrame

import datetime

import time

In [ ]:
import sqlite3

con = sqlite3.connect("news.db") #test.db생성
cur = con.cursor()

cur.executescript("""
        DROP TABLE IF EXISTS table1;
        CREATE TABLE table1(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        link TEXT NOT NULL
        );

        DROP TABLE IF EXISTS table2;
        CREATE TABLE table2(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        date TEXT NOT NULL,
        title TEXT NOT NULL,
        content TEXT NOT NULL,
        link TEXT NOT NULL
        );
""")

con.commit()

In [2]:
def download(method,url,param=None,data=None,timeout=1,maxretries=3):
    try:
        resp=requests.request(method,url,params=param,data=data)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500<=e.response.status_code<600 and maxretries>0 :
            print(maxretries)
            time.sleep(timeout)
            download(method,url,param,data,timeout,maxretries-1)
            print("재시도")
        else:
            print(e.reponse.status_code)
            print(e.response.reason)
    return resp

In [3]:
def news_1page(p_id=264,dates=20190730,pages_num=1):
    url="https://news.naver.com/main/list.nhn?mode=LS2D"
    param={
        'sid2':p_id,
        'sid1':100,
        'mid':'shm',
        'date':dates,
        'page':pages_num,
    }
    html=download('get',url,param=param)
    dom=BeautifulSoup(html.text,'lxml')

    tk=dom.find_all('td',{"class":'content'})[0] .find_all({'a':'href'})  #링크,날짜,페이지 들어간거 찾고 a 그래도 링크,페이지,날짜

    #이미지 중복 링크 제거용도 이미지 없는거에 대한 대비책
    tktk=[]
    for i in range(len(tk)):
        try:
            len(tk[i].img) #이미지를 뽑아냈을때 길이가 있다는거는 이미지 관련 부분
        except:
            tktk.append(list(tk[i].attrs.values())) #이미지 아닌 부분 출력
            

    tktk=[y for x in tktk for y in x] #멀티 리스트 되있는거를 풀어줄때 사용 -> 링크, 페이지랑, 날짜

    #필요없는 날짜부분 제거 길이 방식으로 그냥 제거 날짜부분 길이가 1임을 이용했음
    tktk2=[]
    for i in range(len(tktk)):
        if len(tktk[i])>2:
            tktk2.append(tktk[i])
        else:
            pass

    #링크부분 뽑아내기
    links=[]
    for i in range(len(tktk2)):
        if tktk2[i].find('https')>(-1): #find를 이용해서 https가 없으면은 -1 있으면 -1 초과의 값 0~
            links.append(tktk2[i])
        else:
            pass
    links2=DataFrame(links,columns=['links'])

    #1페이지, 11페이지 21.... 페이지만 뽑아내기
    pages=[]
    for i in range(len(tktk2)):
        if tktk2[i].find('page')>(-1): #find 방식으로 페이지 뽑아내기
            pages.append(tktk2[i][-3:]) #여기서 page=11 page=9
        else:
            pass

    pages=[i.replace('e=','') for i in pages] #replace를 이용해서 
    pages=[i.replace('=','') for i in pages]
    pages=[int(s) for s in pages] #int 바꾸려고 해당 부분 사용

    pages2=[] #페이지 중복 제거 #11 -> 10 12 13 14 ....
    for i in range(len(pages)):
        if pages[i]>pages_num: #제일 위에 페이지 넘버 가지고 와서 그 숫자보다 큰 것만 추가
            pages2.append(pages[i])
        else:
            pass
    pages=[str(s) for s in pages2] #그냥 숫자값으로 들어가면 페이지 부분에서 오류 생겨서 str로 변경
    
    return links,pages
    

In [4]:
def news_other_page(p_id=264,dates=20190730,pages_num=2):
    url="https://news.naver.com/main/list.nhn?mode=LS2D"
    param={
        'sid2':p_id,
        'sid1':100,
        'mid':'shm',
        'date':dates,
        'page':pages_num,
    }
    html=download('get',url,param=param)
    dom=BeautifulSoup(html.text,'lxml')
    tk=dom.find_all('td',{"class":'content'})[0].find_all({'a':'href'})
    
    tktk=[]
    for i in range(len(tk)):
        try:
            len(tk[i].img)
        except:
            tktk.append(list(tk[i].attrs.values()))

    tktk=[y for x in tktk for y in x]

    tktk2=[]
    for i in range(len(tktk)):
        if len(tktk[i])>2:
            tktk2.append(tktk[i])
        else:
            pass
        
    links=[]
    for i in range(len(tktk2)):
        if tktk2[i].find('https')>(-1):
            links.append(tktk2[i])
        else:
            pass
        
    return links

In [5]:
# page_total=[]
# for i in range(10):
#     links2,pages2=news_1page(p_id=265,dates=20190802,pages_num=(10*i)+1)
#     page_total.append(pages2)
# page_total=[y for x in page_total for y in x]

# links_total,pages2=news_1page(p_id=265,dates=20190802,pages_num=1)
# links_total=DataFrame(links_total,columns=['links'])

# for i in range(2,len(page_total)+2):
#     links=DataFrame(news_other_page(p_id=265,dates=20190802,pages_num=i),columns=['links'])
#     links_total=pd.concat([links_total,links],axis=0)
# links_total=links_total.reset_index(drop=True)

In [6]:
# a = datetime.datetime(2019,8,2)
# links_real_total=DataFrame(columns=['links'])
# s_id=264

# for j in range(3):
#     d_time=int((a - datetime.timedelta(days=j)).strftime('%Y%m%d'))
#     page_total=[]
    
#     for i in range(10):
#         links2,pages2=news_1page(p_id=s_id,dates=d_time,pages_num=(10*i)+1)
#         page_total.append(pages2)
#     page_total=[y for x in page_total for y in x]

#     links_total,pages2=news_1page(p_id=s_id,dates=d_time,pages_num=1)
#     links_total=DataFrame(links_total,columns=['links'])

#     for k in range(2,len(page_total)+2):
#         links=DataFrame(news_other_page(p_id=s_id,dates=d_time,pages_num=k),columns=['links'])
#         links_total=pd.concat([links_total,links],axis=0)
#         links_total=links_total.reset_index(drop=True)
    
#     links_real_total=pd.concat([links_real_total,links_total],axis=0)
#     links_real_total=links_real_total.reset_index(drop=True)

In [7]:
a = datetime.datetime(2019,8,7) #원하는 날짜 지정
links_real_total=DataFrame(columns=['links']) #링크 빈 data frame

for m in range(264,270): #264~269 
    for j in range(10): #원하는 날짜 주기 # 1넣으면 당일만 #3 넣으면은 8/2, 8/1 7/31 
        d_time=int((a - datetime.timedelta(days=j)).strftime('%Y%m%d')) #기준 날짜에서 빼서 하는거 8/2 -> 8/1 
        page_total=[] #빈 페이지 번호

        #해당 카테코리 부분의 페이지 번호 전체 가져오기
        for i in range(15): #여유있게 지정해둔 것
            links2,pages2=news_1page(p_id=m,dates=d_time,pages_num=(10*i)+1) #1페이지 -> 11페이지 -> 21 -> 31 .....
            page_total.append(pages2) #페이지가 동일하게 처리 되는 시점 부터는 최종 페이지까지만 나오게 됨
        page_total=[y for x in page_total for y in x] #멀티 리스트 처리

        links_total,pages2=news_1page(p_id=m,dates=d_time,pages_num=1) #링크 처음부터 넣는 부분
        links_total=DataFrame(links_total,columns=['links']) #1페이지에 있는 링크 dataframe
 
        #2페이지부터의 과정
        for k in range(2,len(page_total)+2):
            links=DataFrame(news_other_page(p_id=m,dates=d_time,pages_num=k),columns=['links']) #2페이지 짜리는 바로 링크만 있으니 바로 dataframe
            links_total=pd.concat([links_total,links],axis=0) #concat 해서 dataframe 바로 밑으로 합치기 
            links_total=links_total.reset_index(drop=True)

        links_real_total=pd.concat([links_real_total,links_total],axis=0)
        links_real_total=links_real_total.reset_index(drop=True)

In [33]:
#링크 단계에서 오류가 났을때 일단 집어 넣고 돌릴수 있도록 만들어 놓음
for i in range(len(links_real_total)):
    cur.execute("""
    INSERT INTO table1
    (link)
    VALUES(?)
    """, [links_real_total['links'][i]])
    con.commit()

In [11]:
# link=[]
# date=[]
# title=[]
# write=[]

# # for i in range(len(links_real_total)):
# for i in range(2600,2700):
#     html=download('get',links_real_total['links'][i])
#     dom=BeautifulSoup(html.text,'lxml')
#     time.sleep(0.1)
#     try:
#         oo=dom.find('div',{'class':'sponsor'}).text #정치 기사가 아닌 텍스트 골라내기 용도
#     except:
#         pass
#     else:
#         link.append(links_real_total['links'][i])
#         #date 입력되는 길이 이용해서 걸러내는 방법
#         date.append(oo[dom.find('div',{'class':'sponsor'}).text.find('기사입력')+5:dom.find('div',{'class':'sponsor'}).text.find('기사입력')+25])

#         #title 찾고 추가로 들어가있는 부분 대체하는 방법
#         title.append(dom.find('title').text.replace(" : 네이버 뉴스",""))

#         #본문 일단 다 가져오는 방법 #함수 제거하는 것과 광고 부분을 길이를 이용해서 제거해버림
#         o2=dom.find_all('div',{'id':'articleBodyContents'})[0].text
#         write.append(o2[dom.find_all('div',{'id':'articleBodyContents'})[0].text.find('()')+3:dom.find_all('div',{'id':'articleBodyContents'})[0].text.find('▶')])
        

In [35]:
for i in range(len(links_real_total)):
    html=download('get',links_real_total['links'][i])
    dom=BeautifulSoup(html.text,'lxml')
    time.sleep(0.2) #timeout error 방지용 최소마지노선
    try:
        oo=dom.find('div',{'class':'sponsor'}).text #정치 기사가 아닌 텍스트 골라내기 용도 #정치기사 아닌거 돌리면 무조건 에러남
    except:
        pass #정치기사 아닌건 db에 안 넣을 거임
    else:
        o2=dom.find_all('div',{'id':'articleBodyContents'})[0].text #기사 내용 걸러내기 1차, 밑에는 2차 걸러내기
        content=o2[dom.find_all('div',{'id':'articleBodyContents'})[0].text.find('()')+3:dom.find_all('div',{'id':'articleBodyContents'})[0].text.find('▶')].replace('\xa0','').replace("\'",'').replace('\t','').replace('{}\n\n','')
        if dom.find('title').text.replace(" : 네이버 뉴스","").find('속보')>(-1) or len(content)<350: #속보인거랑 길이 350가 안되는 사진기사나 간략 기사는 걸러낼거임
            pass
        else:
            date=oo[dom.find('div',{'class':'sponsor'}).text.find('기사입력')+5:dom.find('div',{'class':'sponsor'}).text.find('기사입력')+25].replace('\n','').replace('\r','')
            #날짜 부분은 길이를 이용해서 추가하고 삐져나온 부분만 제거해주는 방식으로 함
            
            #주어진 table에다가 넣는 방식으로 하고 이후에 db에서 골라낸 뒤에 다시 집어넣는 방식을 사용하는 것을 생각하고 있음
            cur.execute("""
            INSERT INTO table2
            (date, title, content, link)
            VALUES(?,?,?,?)
            """, [date, dom.find('title').text.replace(" : 네이버 뉴스",""), content, links_real_total['links'][i]] )
            con.commit()
        

하루치 돌아가는데 4900개 기준 28분

In [35]:
# # for i in range(len(links_real_total)):
# for i in range(2600,2700):
#     html=download('get',links_real_total['links'][i])
#     dom=BeautifulSoup(html.text,'lxml')
#     time.sleep(0.1)
#     try:
#         oo=dom.find('div',{'class':'sponsor'}).text #정치 기사가 아닌 텍스트 골라내기 용도 #정치기사 아닌거 돌리면 무조건 에러남
#     except:
#         pass #정치기사 아닌건 db에 안 넣을음
#     else:
#         date=oo[dom.find('div',{'class':'sponsor'}).text.find('기사입력')+5:dom.find('div',{'class':'sponsor'}).text.find('기사입력')+25].replace('\n','').replace('\r','')
        
#         o2=dom.find_all('div',{'id':'articleBodyContents'})[0].text
#         content=o2[dom.find_all('div',{'id':'articleBodyContents'})[0].text.find('()')+3:dom.find_all('div',{'id':'articleBodyContents'})[0].text.find('▶')].replace('\xa0','').replace("\'",'').replace('\t','')
        
#         cur.execute("""
#         INSERT INTO table1
#         (date, title, content, link)
#         VALUES(?,?,?,?)
#         """, [date, dom.find('title').text.replace(" : 네이버 뉴스",""), content, links_real_total['links'][i]] )
#         con.commit()
        